This notebook uses ipython-sql (pip install ipython-sql) to type SQL queries directly. Use %sql before a single-line query and %%sql before a multi-line query.

In [1]:
%load_ext sql

In [16]:
%sql postgresql://nwespe@localhost/birth_db

u'Connected: nwespe@birth_db'

In [18]:
# use SET search_path TO schema_name if schema is used
%sql reset search_path

Done.


[]

In [19]:
%%sql
SELECT * FROM birth_data_table # * means select all columns
LIMIT 10  # only return 10 rows
OFFSET 50  # start at row 50

10 rows affected.


index,alcohol_use,anencephaly,attendant,birth_loc_type,birth_month,birth_state,birth_weight,birth_year,cigarette_use,cigarettes_per_day,cigarettes_trimester1,cigarettes_trimester2,cigarettes_trimester3,day,delivery_method,downs syndrome,drinks_per_week,father_age,father_race,gestation_weeks,infant_sex,mother_age,mother_birth_country,mother_birth_state,mother_education,mother_marital_status,mother_race,mother_state,population,pregnancy_weight,resident,revision,spina_bifida,table,timestamp,uses_tobacco,weight_gain
50,None,None,MD,None,Jan,None,5000.0,2012,None,None,None,None,None,Fri,Vaginal,None,None,None,None,42.0,F,20,None,None,None,Yes,White,None,None,None,Resident,S,None,births12.txt,1325456800,None,30.0
51,None,None,MD,None,Jan,None,3000.0,2012,None,None,None,None,None,Fri,Vaginal,None,None,None,None,42.0,F,20,None,None,None,Yes,White,None,None,None,Resident,S,None,births12.txt,1326371490,None,32.0
52,None,None,MD,None,Jan,None,3000.0,2012,None,None,None,None,None,Sun,Vaginal,None,None,None,None,38.0,F,21,None,None,None,No,White,None,None,None,Resident,S,None,births12.txt,1327245443,None,24.0
53,None,None,MD,None,Feb,None,4000.0,2012,None,None,None,None,None,Wed,Cesarean,None,None,None,None,39.0,M,35,None,None,None,Yes,White,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1328148996,None,30.0
54,None,None,MD,None,Feb,None,3000.0,2012,None,None,None,None,None,Wed,Vaginal,None,None,None,None,38.0,F,29,None,None,None,Yes,White,None,None,None,Resident,S,None,births12.txt,1328933795,None,34.0
55,None,None,MD,None,Feb,None,2000.0,2012,None,None,None,None,None,Sat,Vaginal,None,None,None,None,31.0,M,27,None,None,None,No,Black,None,None,None,Intra-State/Territor Non-resident (diff county),S,None,births12.txt,1329753116,None,25.0
56,None,None,MD,None,Mar,None,4000.0,2012,None,None,None,None,None,Tu,Vaginal,None,None,None,None,40.0,F,18,None,None,None,No,White,None,None,None,Resident,S,None,births12.txt,1330659032,None,50.0
57,None,None,MD,None,Mar,None,4000.0,2012,None,None,None,None,None,Sun,Vaginal,None,None,None,None,38.0,F,29,None,None,None,Yes,Black,None,None,None,Resident,S,None,births12.txt,1331541002,None,50.0
58,None,None,MD,None,Mar,None,4000.0,2012,None,None,None,None,None,Mon,Vaginal,None,None,None,None,39.0,F,29,None,None,None,No,White,None,None,None,Resident,S,None,births12.txt,1332469424,None,31.0
59,None,None,MD,None,Apr,None,4000.0,2012,None,None,None,None,None,Sat,Vaginal,None,None,None,None,39.0,M,30,None,None,None,No,White,None,None,None,Resident,S,None,births12.txt,1333389308,None,3.0


What is the most common day for a delivery?

Let's first create a view containing the days of the week and the count for each day

In [43]:
%%sql 
CREATE VIEW day_counts AS
SELECT day, COUNT(*) AS count
FROM birth_data_table
GROUP BY day

(psycopg2.ProgrammingError) relation "day_counts" already exists
 [SQL: 'CREATE VIEW day_counts AS\nSELECT day, COUNT(*) AS count\nFROM birth_data_table\nGROUP BY day']


In [44]:
%sql SELECT * FROM day_counts

7 rows affected.


day,count
Tu,449
Sat,307
Mon,403
Wed,415
Sun,281
Th,432
Fri,429


We can view this table ordered by count.

In [46]:
%%sql 
SELECT * 
FROM day_counts 
ORDER BY count DESC  # order in descending

7 rows affected.


day,count
Tu,449
Th,432
Fri,429
Wed,415
Mon,403
Sat,307
Sun,281


For a solution with one query and one sub-query, see below. Note that this solution creates a temporary table instead of a view.

In [62]:
%%sql
SELECT day
FROM (SELECT day, COUNT(*) AS count  # the parentheses enclose a subquery
      FROM birth_data_table
      GROUP BY day
     ) AS day_counts  # name the temporary table
WHERE count = (SELECT MAX(count) FROM day_counts)

1 rows affected.


day
Tu
